In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)
import folium
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from glob import glob
# 파일 갯수 확인
len(glob('./과제data/소상공인*.csv'))

17

In [3]:
all_shop = glob('./과제data/소상공인*.csv')

In [18]:
df = pd.read_csv(all_shop[1], sep='|')
df['상호명'] = df['상호명'].str.lower()

In [24]:
df[df['상호명'].str.contains('롯데리아')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [4]:
df = pd.read_csv(all_shop[8], sep='|')
df['상호명'] = df['상호명'].str.lower()

In [15]:
df[df['상호명'].str.contains('맘스터치')]

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
224636,11786061,moms헤어,NaN,F,생활서비스,F01,이/미용/건강,F01A01,여성미용실,S96112,...,1153010200103290081023762,NaN,서울특별시 구로구 구로동로28길 39,152050,8308.0,NaN,1,NaN,126.886364,37.490909
250577,12281721,tomamoms커피,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1174010900104530021010314,원방빌딩,서울특별시 강동구 천호대로157길 18,134864,5335.0,NaN,NaN,NaN,127.127466,37.538485


In [37]:
import cartogram.fillter0 as ft


In [38]:
c_d = ft.fillter(all_shop)

In [40]:
c_p = pd.DataFrame(c_d)

In [41]:
c_p.rename({0:'ID', 1:'버거킹', 2: '맥도날드', 3:'KFC', 4:'롯데리아', 5:'맘스터치'},axis=1 ,inplace=True)

In [42]:
c_p['버거지수'] = (c_p['버거킹'] + c_p['맥도날드'] + c_p['KFC']) / (c_p['롯데리아'] + c_p['맘스터치'])

In [43]:
c_p

,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수
0,홍천,4,0,0,3,1,1.000000
1,강릉,1,1,0,5,4,0.222222
2,영월,0,0,0,0,0,NaN
3,원주,1,5,0,13,4,0.352941
4,삼척,0,0,0,0,1,0.000000
...,...,...,...,...,...,...,...
245,괴산,0,0,0,1,0,0.000000
246,증평,0,0,0,1,1,0.000000
247,영동,0,0,0,0,0,NaN
248,단양,0,0,0,0,0,NaN


In [44]:
c_p[c_p['버거지수'] == np.inf]

,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수
9,동해,0,3,1,0,0,inf
72,통영,0,2,0,0,0,inf
80,밀양,0,2,0,0,0,inf


In [45]:
c_p.replace([np.inf], 0,inplace=True)

In [46]:
c_p = c_p.fillna(0)

In [48]:
draw_korea_raw = pd.read_excel('./과제data/draw_korea_raw(수정).xlsx')
draw_korea_raw_stacked = pd.DataFrame(draw_korea_raw.stack())
draw_korea_raw_stacked.reset_index(inplace=True)
draw_korea_raw_stacked.rename(columns={'level_0' : 'y', 'level_1' : 'x', 0: 'ID'}, inplace=True)
draw_korea = draw_korea_raw_stacked
draw_korea.loc[draw_korea['ID'] == '인천 남구', 'ID'] = '인천 미추홀'

In [49]:
c_p = pd.merge(c_p, draw_korea, how='left', on=['ID'])
c_p.head()

,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수,y,x
0,홍천,4,0,0,3,1,1.000000,2,8
1,강릉,1,1,0,5,4,0.222222,3,11
2,영월,0,0,0,0,0,0.000000,7,10
3,원주,1,5,0,13,4,0.352941,5,10
4,삼척,0,0,0,0,1,0.000000,5,11


In [50]:
c_p.to_csv('./static/data/b_p.csv')

In [16]:
df = pd.read_csv('./static/data/b_p.csv')

In [17]:
df[df['ID'] == '광명']

,Unnamed: 0,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수,y,x
54,54,광명,1,5,1,1,0,7.0,4,4


In [7]:
c_p